In [1]:
import google.generativeai as genai

c:\Users\ADMIN\anaconda3\envs\aider\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=[add, subtract, multiply, divide]
)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x0000028173618310>,
    system_instruction=None,
    cached_content=None
)

In [3]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [6]:
response = chat.send_message(
    "trong túi có 16 cái kẹo chia đều cho mỗi người 4 cái kẹo hỏi số người nhận được 4 cái kẹo"
)
response.text

'Số người nhận được kẹo là: 16 : 4 = 4 (người)\n\nVậy có **4 người** nhận được kẹo. \n'

In [7]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
model -> [{'text': "That's 2508 mittens in total. \n"}]
user -> [{'text': 'trong túi có 18 cái kẹo chia đều cho mỗi người 4 cái kẹo hỏi số người nhận được 4 cái kẹo và có dư kẹo không'}]
model -> [{'text': 'Số người nhận được kẹo là: 18 : 4 = 4 (người) dư 2 cái kẹo\n\nVậy có **4 người** nhận được kẹo và **dư 2 cái kẹo**. \n'}]
user -> [{'text': 'trong túi có 16 cái kẹo chia đều cho mỗi người 4 cái kẹo hỏi số người nhận được 4 cái kẹo'}]
model -> [{'text': 'Số người nhận được kẹo là: 16 : 4 = 4 (người)\n\nVậy có **4 người** nhận được kẹo. \n'}]


In [11]:
def find_movies(description: str, location: str = ""):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]

In [12]:
functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=functions.values())

In [14]:
response = model.generate_content(
    "Which theaters in Mountain View show the Barbie movie?"
)
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}
]

In [15]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)


part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']
